In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

In [16]:
df = pd.read_csv("Data_processed/data.csv")

In [17]:
df.head()

,customer_lat,customer_lng,order_status,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_score,...,product_length_cm,product_height_cm,product_width_cm,seller_lat,seller_lng,order_time,recency,amount,frequency,product_category
0,-23.499063,-46.767939,1.0,89.80,24.94,1,0,8,114.74,1,...,46.0,39.0,22.0,-23.049552,-47.837621,5.0,287,89.800000,1,37
1,-20.794069,-49.365081,1.0,84.80,24.90,1,0,2,109.70,4,...,46.0,39.0,22.0,-23.049552,-47.837621,8.0,361,84.800000,1,37
2,-23.009475,-43.334530,1.0,89.80,18.00,1,0,2,107.80,4,...,46.0,39.0,22.0,-23.049552,-47.837621,25.0,316,89.800000,1,37
3,-23.590310,-46.723572,1.0,84.80,14.04,1,0,3,291.47,4,...,46.0,39.0,22.0,-23.049552,-47.837621,18.0,372,83.896667,3,37
4,-23.590310,-46.723572,1.0,96.99,9.36,1,0,3,291.47,4,...,48.0,27.0,18.0,-23.049552,-47.837621,18.0,372,83.896667,3,37


In [ ]:
x = []
y = []
for n in range(1,10):
    x.append(n)
    estimator = make_pipeline(StandardScaler(), KMeans(n_clusters=n))
    estimator.fit(df)
    y.append(estimator.inertia_)
pl